In [ ]:
# 1. Load data: passage_embeddings, query_train_embeddings, train_data (a proportion of)
# 2. Network input: 768 output: 32
# 3. Train: concatenate original 768 with 32 

## Import data: load_data.py

In [15]:
import csv
import pickle
import numpy as np

PASSAGE_NP_PATH = "/home/jianx/results/passage_0__emb_p__data_obj_0.pb"
PASSAGE_MAP_PATH = "/datadrive/jianx/data/annoy/100_ance_passage_map.dict"
QUERY_TRAIN_NP_PATH = "/home/jianx/results/query_0__emb_p__data_obj_0.pb"
QUERY_MAP_PATH = "/datadrive/jianx/data/annoy/100_ance_query_train_map.dict"
# TRAIN_RANK_PATH = "/datadrive/jianx/data/train_data/ance_training_rank100_nqueries250000_200000_random_Aug_24_19:43:52.csv"
# TRAIN_RANK_PATH = "/datadrive/ruohan/reverse_ranker/new_training/combine_rank_train_phase2.csv"
# TRAIN_RANK_PATH = "/datadrive/jianx/data/train_data/ance_training_rank100_nqueries50000_200000_Aug_26_20:59:48.csv"
# New two step random sampling
# TRAIN_RANK_PATH = "/datadrive/jianx/data/train_data/ance_training_rank100_nqueries50000_200000_Sep_03_22:56:31.csv"
TRAIN_RANK_PATH = "/datadrive/ruohan/data/rerank_learnBM25_morepos.csv"
# TRAIN_RANK_PATH = "/datadrive/ruohan/data/rerank_full_query_log_train.csv"

OUT_RANK = 200
N_PASSAGE = 200000
def obj_reader(path):
    with open(path, 'rb') as handle:
        return pickle.load(handle, encoding="bytes")
def load_train(path):
    with open(path, "r") as file:
        pos_dict = {}
        neg_dict = {}
        count = 0
        for line in file:
#             if count >= N_PASSAGE * 100:
#                 break
#             count += 1
            tokens = line.split(",")
            pid = int(tokens[0])
            qid = int(tokens[1])
            rank = int(tokens[2].rstrip())
            if rank == 0:
                if pid not in neg_dict:
                    neg_dict[pid] = {}
                neg_dict[pid][qid] = OUT_RANK
            else:
                if pid not in pos_dict:
                    pos_dict[pid] = {}
                pos_dict[pid][qid] = rank
    return pos_dict, neg_dict
def map_id(old_np, mapping):
    new_dict = dict(zip(mapping.values(),old_np))
    return new_dict
def load():
    print("Load embeddings.")
    passage_np = obj_reader(PASSAGE_NP_PATH)
    pid_mapping = obj_reader(PASSAGE_MAP_PATH)
    query_np = obj_reader(QUERY_TRAIN_NP_PATH)
    qid_mapping = obj_reader(QUERY_MAP_PATH)
    print("Mapping ids.")
    query_dict = map_id(query_np, qid_mapping)
    passage_dict = map_id(passage_np, pid_mapping)
    print("Load training data.")
    train_pos_dict, train_neg_dict = load_train(TRAIN_RANK_PATH)
    return train_pos_dict, train_neg_dict, query_dict, passage_dict

In [16]:
train_pos_dict, train_neg_dict = load_train(TRAIN_RANK_PATH)

In [19]:
len(train_neg_dict)

49749

In [6]:
train_pos_dict, train_neg_dict, query_dict, passage_dict = load()

Load embeddings.
Mapping ids.
Load training data.


In [29]:
# def obj_writer(obj, path):
#     with open(path, 'wb') as handle:
#         pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
# obj_writer(train_pos_dict, "/datadrive/ruohan/data/train_pos_dict.pb")
# obj_writer(train_neg_dict, "/datadrive/ruohan/data/train_neg_dict.pb")

## Network Architecture: network.py

In [25]:
import torch
import torch.nn as nn

NUM_HIDDEN_NODES = 1536
NUM_HIDDEN_LAYERS = 3
DROPOUT_RATE = 0.1
    
# Define the network
class ResidualNet(torch.nn.Module):

    def __init__(self, embed_size):
        super(ResidualNet, self).__init__()
        
        self.input = nn.Linear(embed_size, NUM_HIDDEN_NODES)
        self.relu = nn.ReLU()
        self.normlayer = nn.LayerNorm(NUM_HIDDEN_NODES)
        self.dropout = nn.Dropout(p=DROPOUT_RATE)
        self.output = nn.Linear(NUM_HIDDEN_NODES, embed_size)

    def forward(self, x):
        identity = x
        out = x
        for i in range(NUM_HIDDEN_LAYERS):
            out = self.input(out)
            out = self.relu(out)
            out = self.normlayer(out)
            out = self.dropout(out)
            out = self.output(out)
            out += identity
#             out = self.relu(out)
        return out

    def parameter_count(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

## Train reverse ranker: train.py

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import torch.nn.functional as F


TOP_K = 100

# With probability alpha
# Select a random negative sample from train_neg_dict
ALPHA = 0.5

def dot_product(A, B, normalize=False):
    if normalize:
        A = F.normalize(A)
        B = F.normalize(B)
    b = A.shape[0]
    embed = A.shape[1]
    result = torch.bmm(A.view(b, 1, embed), B.view(b, embed, 1))
    return result


def mini_batch(batch_size, device, train_pos_dict, train_neg_dict, query_dict, passage_dict):
    passage_list = list(train_neg_dict.keys())
    passages = []
    pos = []
    neg = []
    pos_rank_list = []
    neg_rank_list = []
    while len(passages) < batch_size:
        pid = random.sample(passage_list, 1)[0]
        try:
            temp_pos_list = list(train_pos_dict[pid].keys())
        except:
            continue
        try:
            temp_neg_list = list(train_neg_dict[pid].keys())
        except:
            continue
        if np.random.uniform(0,1,1) <= ALPHA:
            random_positive = random.sample(temp_pos_list, 1)
            pos_qid = random_positive[0]
            pos_rank = train_pos_dict[pid][pos_qid]
            random_negative = random.sample(temp_neg_list, 1)
            neg_qid = random_negative[0]
            neg_rank = train_neg_dict[pid][neg_qid]
#             not_negative = True
#             while not_negative:
#                 temp_neg_qid = random.sample(list(query_dict.keys()), 1)
#                 if temp_neg_qid not in temp_query_list:
#                     neg_qid = temp_neg_qid[0]
#                     neg_rank = 1000
#                     not_negative = False
        else:
            if len(temp_pos_list) < 2:
                continue
            pos_neg_pair = random.sample(temp_pos_list, 2)
            # e.g. 60 >= 3
            if train_pos_dict[pid][pos_neg_pair[0]] >= train_pos_dict[pid][pos_neg_pair[1]]:
                pos_qid = pos_neg_pair[1]
                neg_qid = pos_neg_pair[0]
            # e.g. 3 < 60
            else:
                pos_qid = pos_neg_pair[0]
                neg_qid = pos_neg_pair[1]   
            pos_rank = train_pos_dict[pid][pos_qid]
            neg_rank = train_pos_dict[pid][neg_qid]
        p_seq = query_dict[pid]
        pos_seq = passage_dict[pos_qid]
        neg_seq = passage_dict[neg_qid]
        passages.append(p_seq)
        pos.append(pos_seq)
        neg.append(neg_seq)
        pos_rank_list.append(TOP_K - pos_rank)
        neg_rank_list.append(TOP_K - neg_rank)
#         pos_rank_list.append((TOP_K - pos_rank) * 2)
#         neg_rank_list.append((TOP_K - neg_rank) * 2)
    labels = torch.stack([torch.FloatTensor(pos_rank_list), torch.FloatTensor(neg_rank_list)], dim=1)
    passages = torch.from_numpy(np.stack(passages))
    pos = torch.from_numpy(np.stack(pos))
    neg = torch.from_numpy(np.stack(neg))
    return passages.to(device), pos.to(device), neg.to(device), labels.to(device)


def train(net, epoch_size, batch_size, optimizer, device, train_pos_dict, train_neg_dict, 
          query_dict, passage_dict, scale=10, loss_option="ce"):
    bce = nn.BCELoss()
    ce = nn.CrossEntropyLoss()
    softmax = nn.Softmax(dim=1)
    train_loss = 0.0
    net.train()
    for mb_idx in range(epoch_size):
        # Read in a new mini-batch of data!
        passages, pos, neg, labels = mini_batch(batch_size, device, train_pos_dict, train_neg_dict, 
                                                query_dict, passage_dict)
        optimizer.zero_grad()
        p_embed = net(passages).to(device)
        pos_embed = net(pos).to(device)
        neg_embed = net(neg).to(device)
        out_pos = dot_product(p_embed, pos_embed).to(device)
        out_neg = dot_product(p_embed, neg_embed).to(device)
        out = torch.cat((out_pos, out_neg), -1).squeeze()
        if loss_option == "bce":
            loss = bce(softmax(out), softmax(labels))
        if loss_option == "ce":
            loss = ce(out, torch.tensor([0 for i in range(batch_size)]).to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        # print(str(mb_idx) + " iteration: " + str(train_loss / (mb_idx + 1)))
    return train_loss / epoch_size


## Main function: main.py

In [22]:
import datetime
from datetime import datetime, timezone, timedelta

TIME_OFFSET = -4


def print_message(s, offset=TIME_OFFSET):
    print("[{}] {}".format(datetime.now(timezone(timedelta(hours=offset))).strftime("%b %d, %H:%M:%S"), s), flush=True)

In [26]:
import torch
from torch import optim
import csv
import sys
import os


MODEL_PATH = "/datadrive/ruohan/rerank/learn_BM25/"
CURRENT_DEVICE = "cuda:2"
PRETRAINED_PATH = "/datadrive/ruohan/random_sample/reverse_alpha0.5_initial_residual_saveoptim_layer51000_100_1000_0.0001_768.model"

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)


def main(num_epochs, epoch_size, batch_size, learning_rate, model_path, embed_size, pretrained=False):
    if pretrained:
        checkpoint = torch.load(PRETRAINED_PATH)
        net = ResidualNet(embed_size=embed_size)
        net.load_state_dict(checkpoint['model'])
        net.to(CURRENT_DEVICE)
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)
        optimizer.load_state_dict(checkpoint['optimizer'])
    else:
        net = ResidualNet(embed_size=embed_size).to(CURRENT_DEVICE)
        optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    print("Loading data")
#     train_rank_dict, query_dict, passage_dict = load()
#     print("Data successfully loaded.")
#     print("Positive Negative Pair dict size: " + str(len(train_rank_dict)))
#     print("Num of queries: " + str(len(query_dict)))
#     print("Num of passages: " + str(len(passage_dict)))
#     print("Finish loading.")

    arg_str = "reverse_alpha0.5_layer3_residual_add_pos" + str(num_epochs) + "_" + str(epoch_size) + "_" + str(batch_size) + "_" + str(learning_rate) + "_" + str(
        embed_size)
    unique_path = model_path + arg_str + ".model"
    output_path = model_path + arg_str + ".csv"
    
    for ep_idx in range(num_epochs):
        train_loss = train(net, epoch_size, batch_size, optimizer, CURRENT_DEVICE, train_pos_dict, 
                           train_neg_dict, query_dict, passage_dict)
        print_message([ep_idx,train_loss])
        with open(output_path, mode='a+') as output:
            output_writer = csv.writer(output)
            output_writer.writerow([ep_idx, train_loss])
        torch.save({
                "model": net.state_dict(),
                "optimizer": optimizer.state_dict(),
                "n_epoch": ep_idx,
                "train_loss": train_loss,
                "n_hidden_layer": NUM_HIDDEN_LAYERS
                    }, unique_path)

In [27]:
# checkpoint = torch.load("/datadrive/ruohan/random_sample/reverse_alpha0.5_initial_residual_saveoptim1000_100_1000_0.0001_768.model")
# ttnet = ResidualNet(embed_size=768)
# ttoptimizer = optim.Adam(ttnet.parameters(), lr=0.0001)
# ttnet.load_state_dict(checkpoint["model"])
# ttoptimizer.load_state_dict(checkpoint["optimizer"])

In [28]:
main(1000,100,1000,0.0001,MODEL_PATH,768,pretrained=False)

Loading data
[Sep 10, 17:16:52] [0, 2.6534855341911316]
[Sep 10, 17:16:58] [1, 2.0709690010547637]
[Sep 10, 17:17:04] [2, 1.9588668596744538]
[Sep 10, 17:17:10] [3, 1.874799748659134]
[Sep 10, 17:17:16] [4, 1.819699022769928]
[Sep 10, 17:17:22] [5, 1.7735561430454254]
[Sep 10, 17:17:28] [6, 1.7268571281433105]
[Sep 10, 17:17:35] [7, 1.6794574213027955]
[Sep 10, 17:17:41] [8, 1.6174738812446594]
[Sep 10, 17:17:47] [9, 1.589144334793091]
[Sep 10, 17:17:53] [10, 1.5339120864868163]
[Sep 10, 17:17:59] [11, 1.4910863149166107]
[Sep 10, 17:18:05] [12, 1.4505127680301666]
[Sep 10, 17:18:11] [13, 1.4103055250644685]
[Sep 10, 17:18:18] [14, 1.377538151741028]
[Sep 10, 17:18:24] [15, 1.334400590658188]
[Sep 10, 17:18:30] [16, 1.3003819966316223]
[Sep 10, 17:18:36] [17, 1.2590808880329132]
[Sep 10, 17:18:42] [18, 1.2362850046157836]
[Sep 10, 17:18:49] [19, 1.1910551691055298]
[Sep 10, 17:18:55] [20, 1.1499135982990265]
[Sep 10, 17:19:01] [21, 1.1180347251892089]
[Sep 10, 17:19:07] [22, 1.08917027

[Sep 10, 17:35:49] [185, 0.6257273936271668]
[Sep 10, 17:35:55] [186, 0.6209891790151596]
[Sep 10, 17:36:01] [187, 0.6189154058694839]
[Sep 10, 17:36:07] [188, 0.6221991646289825]
[Sep 10, 17:36:14] [189, 0.6224400871992111]
[Sep 10, 17:36:20] [190, 0.6221502804756165]
[Sep 10, 17:36:26] [191, 0.6196722161769866]
[Sep 10, 17:36:32] [192, 0.6199394881725311]
[Sep 10, 17:36:38] [193, 0.6213718926906586]
[Sep 10, 17:36:45] [194, 0.618445515036583]
[Sep 10, 17:36:51] [195, 0.6214503926038742]
[Sep 10, 17:36:58] [196, 0.6196558111906052]
[Sep 10, 17:37:04] [197, 0.6211334389448165]
[Sep 10, 17:37:11] [198, 0.6207108271121978]
[Sep 10, 17:37:17] [199, 0.6222633159160614]
[Sep 10, 17:37:24] [200, 0.6201756191253662]
[Sep 10, 17:37:30] [201, 0.620123193860054]
[Sep 10, 17:37:37] [202, 0.6190417158603668]
[Sep 10, 17:37:43] [203, 0.619708725810051]
[Sep 10, 17:37:49] [204, 0.6193310391902923]
[Sep 10, 17:37:55] [205, 0.6199136531352997]
[Sep 10, 17:38:01] [206, 0.619362719655037]
[Sep 10, 17:38

[Sep 10, 17:54:42] [368, 0.5979705929756165]
[Sep 10, 17:54:49] [369, 0.5992348146438599]
[Sep 10, 17:54:55] [370, 0.599551191329956]
[Sep 10, 17:55:01] [371, 0.5991638487577439]
[Sep 10, 17:55:07] [372, 0.5977526587247849]
[Sep 10, 17:55:14] [373, 0.5987279611825943]
[Sep 10, 17:55:21] [374, 0.5946630853414535]
[Sep 10, 17:55:27] [375, 0.597007377743721]
[Sep 10, 17:55:33] [376, 0.601118351817131]
[Sep 10, 17:55:39] [377, 0.5977081102132797]
[Sep 10, 17:55:45] [378, 0.5982039475440979]
[Sep 10, 17:55:51] [379, 0.5941885650157929]
[Sep 10, 17:55:57] [380, 0.5960462826490402]
[Sep 10, 17:56:04] [381, 0.5979796260595321]
[Sep 10, 17:56:10] [382, 0.595815811753273]
[Sep 10, 17:56:16] [383, 0.5989171928167343]
[Sep 10, 17:56:23] [384, 0.5958823996782303]
[Sep 10, 17:56:29] [385, 0.5980740636587143]
[Sep 10, 17:56:35] [386, 0.5958389729261399]
[Sep 10, 17:56:41] [387, 0.5948500919342041]
[Sep 10, 17:56:47] [388, 0.5958042323589325]
[Sep 10, 17:56:53] [389, 0.5960383880138397]
[Sep 10, 17:56

[Sep 10, 18:13:36] [551, 0.58029303252697]
[Sep 10, 18:13:42] [552, 0.5797852057218552]
[Sep 10, 18:13:48] [553, 0.5831352066993714]
[Sep 10, 18:13:54] [554, 0.5797837918996811]
[Sep 10, 18:14:00] [555, 0.580520151257515]
[Sep 10, 18:14:06] [556, 0.5809553080797195]
[Sep 10, 18:14:12] [557, 0.5823794442415238]
[Sep 10, 18:14:18] [558, 0.5796560674905777]
[Sep 10, 18:14:25] [559, 0.5798747235536575]
[Sep 10, 18:14:31] [560, 0.5791244447231293]
[Sep 10, 18:14:37] [561, 0.578859549164772]
[Sep 10, 18:14:43] [562, 0.577617968916893]
[Sep 10, 18:14:49] [563, 0.5818312054872513]
[Sep 10, 18:14:56] [564, 0.5794953030347824]
[Sep 10, 18:15:02] [565, 0.5794003701210022]
[Sep 10, 18:15:08] [566, 0.5790143674612045]
[Sep 10, 18:15:14] [567, 0.5806053400039672]
[Sep 10, 18:15:20] [568, 0.5800020933151245]
[Sep 10, 18:15:27] [569, 0.5785059702396392]
[Sep 10, 18:15:33] [570, 0.5791634881496429]
[Sep 10, 18:15:39] [571, 0.5811047273874282]
[Sep 10, 18:15:45] [572, 0.5799942719936371]
[Sep 10, 18:15:

[Sep 10, 18:32:49] [734, 0.5680093538761138]
[Sep 10, 18:32:55] [735, 0.5649208229780197]
[Sep 10, 18:33:01] [736, 0.5672132462263108]
[Sep 10, 18:33:07] [737, 0.5690227991342545]
[Sep 10, 18:33:13] [738, 0.5668480169773101]
[Sep 10, 18:33:20] [739, 0.5655765509605408]
[Sep 10, 18:33:26] [740, 0.566342556476593]
[Sep 10, 18:33:33] [741, 0.5655443769693375]
[Sep 10, 18:33:39] [742, 0.5654248088598252]
[Sep 10, 18:33:45] [743, 0.5661850774288177]
[Sep 10, 18:33:51] [744, 0.5669337642192841]
[Sep 10, 18:33:57] [745, 0.5646578085422516]
[Sep 10, 18:34:03] [746, 0.5666878259181977]
[Sep 10, 18:34:09] [747, 0.5653489428758621]
[Sep 10, 18:34:16] [748, 0.5640154123306275]
[Sep 10, 18:34:22] [749, 0.5638819468021393]
[Sep 10, 18:34:28] [750, 0.5664121168851852]
[Sep 10, 18:34:35] [751, 0.5666630643606186]
[Sep 10, 18:34:41] [752, 0.566588294506073]
[Sep 10, 18:34:47] [753, 0.5656574493646622]
[Sep 10, 18:34:53] [754, 0.5648786854743958]
[Sep 10, 18:34:59] [755, 0.5667607164382935]
[Sep 10, 18:

[Sep 10, 18:51:41] [917, 0.5571733331680297]
[Sep 10, 18:51:47] [918, 0.555947658419609]
[Sep 10, 18:51:53] [919, 0.554646720290184]
[Sep 10, 18:51:59] [920, 0.5560131162405014]
[Sep 10, 18:52:06] [921, 0.556145943403244]
[Sep 10, 18:52:12] [922, 0.5570351654291152]
[Sep 10, 18:52:18] [923, 0.5546270292997361]
[Sep 10, 18:52:25] [924, 0.5542134690284729]
[Sep 10, 18:52:31] [925, 0.5567965877056121]
[Sep 10, 18:52:37] [926, 0.5559780484437943]
[Sep 10, 18:52:43] [927, 0.5582089251279831]
[Sep 10, 18:52:49] [928, 0.5583788704872131]
[Sep 10, 18:52:55] [929, 0.5544932436943054]
[Sep 10, 18:53:01] [930, 0.5547147482633591]
[Sep 10, 18:53:07] [931, 0.5563486111164093]
[Sep 10, 18:53:14] [932, 0.5555798453092575]
[Sep 10, 18:53:20] [933, 0.5548386657238007]
[Sep 10, 18:53:26] [934, 0.554086594581604]
[Sep 10, 18:53:33] [935, 0.5558962440490722]
[Sep 10, 18:53:39] [936, 0.556238666176796]
[Sep 10, 18:53:45] [937, 0.5576039880514145]
[Sep 10, 18:53:51] [938, 0.555471979379654]
[Sep 10, 18:53:5

In [12]:
# Reranking training
# New sample strategy: with even more positives
# Number of training queries: 50000
# 1 hidden layers with last relu deleted
main(1000,100,1000,0.0001,MODEL_PATH,768,pretrained=False)

Loading data
[Sep 10, 14:42:29] [0, 1.3513387095928193]
[Sep 10, 14:42:36] [1, 1.0813941437005996]
[Sep 10, 14:42:42] [2, 1.0287969195842743]
[Sep 10, 14:42:48] [3, 0.9800474685430527]
[Sep 10, 14:42:53] [4, 0.9468290579319]
[Sep 10, 14:42:59] [5, 0.915119372010231]
[Sep 10, 14:43:05] [6, 0.8924523371458054]
[Sep 10, 14:43:10] [7, 0.8671925920248031]
[Sep 10, 14:43:15] [8, 0.8424159663915635]
[Sep 10, 14:43:20] [9, 0.8256917613744735]
[Sep 10, 14:43:25] [10, 0.7970497304201126]
[Sep 10, 14:43:30] [11, 0.7943322944641114]
[Sep 10, 14:43:35] [12, 0.769710858464241]
[Sep 10, 14:43:39] [13, 0.7509150296449661]
[Sep 10, 14:43:44] [14, 0.729878979921341]
[Sep 10, 14:43:49] [15, 0.7155831336975098]
[Sep 10, 14:43:54] [16, 0.7070443004369735]
[Sep 10, 14:43:59] [17, 0.6916442793607712]
[Sep 10, 14:44:03] [18, 0.6722440671920776]
[Sep 10, 14:44:08] [19, 0.6469973140954971]
[Sep 10, 14:44:12] [20, 0.6347371101379394]
[Sep 10, 14:44:17] [21, 0.6132521575689316]
[Sep 10, 14:44:21] [22, 0.589971612

[Sep 10, 14:55:43] [183, 0.3822659581899643]
[Sep 10, 14:55:48] [184, 0.3875804555416107]
[Sep 10, 14:55:52] [185, 0.38157801926136015]
[Sep 10, 14:55:56] [186, 0.38550141513347624]
[Sep 10, 14:56:01] [187, 0.3824757394194603]
[Sep 10, 14:56:05] [188, 0.38589973747730255]
[Sep 10, 14:56:09] [189, 0.38327767997980117]
[Sep 10, 14:56:13] [190, 0.38327124118804934]
[Sep 10, 14:56:18] [191, 0.38159925818443297]
[Sep 10, 14:56:22] [192, 0.38257368296384814]
[Sep 10, 14:56:26] [193, 0.382380864918232]
[Sep 10, 14:56:30] [194, 0.3866045492887497]
[Sep 10, 14:56:34] [195, 0.38078167229890825]
[Sep 10, 14:56:38] [196, 0.386432438492775]
[Sep 10, 14:56:42] [197, 0.3824298787117004]
[Sep 10, 14:56:46] [198, 0.38600499391555787]
[Sep 10, 14:56:51] [199, 0.38415184497833255]
[Sep 10, 14:56:55] [200, 0.3851358079910278]
[Sep 10, 14:56:59] [201, 0.38503098368644717]
[Sep 10, 14:57:04] [202, 0.38203520864248275]
[Sep 10, 14:57:08] [203, 0.38297499775886534]
[Sep 10, 14:57:12] [204, 0.3835223886370659]

[Sep 10, 15:08:23] [364, 0.3761520516872406]
[Sep 10, 15:08:27] [365, 0.37886799812316896]
[Sep 10, 15:08:32] [366, 0.3764519160985947]
[Sep 10, 15:08:36] [367, 0.37685601353645326]
[Sep 10, 15:08:40] [368, 0.3741794779896736]
[Sep 10, 15:08:44] [369, 0.37893093258142474]
[Sep 10, 15:08:48] [370, 0.37829080104827884]
[Sep 10, 15:08:53] [371, 0.375488908290863]
[Sep 10, 15:08:57] [372, 0.3780738377571106]
[Sep 10, 15:09:02] [373, 0.3781511569023132]
[Sep 10, 15:09:07] [374, 0.37682812094688417]
[Sep 10, 15:09:11] [375, 0.37766686379909514]
[Sep 10, 15:09:15] [376, 0.3785788881778717]
[Sep 10, 15:09:20] [377, 0.3766642218828201]
[Sep 10, 15:09:24] [378, 0.3775282472372055]
[Sep 10, 15:09:28] [379, 0.37968139618635177]
[Sep 10, 15:09:33] [380, 0.37647779017686844]
[Sep 10, 15:09:37] [381, 0.37756425499916074]
[Sep 10, 15:09:41] [382, 0.3779557803273201]
[Sep 10, 15:09:46] [383, 0.3784627303481102]
[Sep 10, 15:09:50] [384, 0.37529554635286333]
[Sep 10, 15:09:54] [385, 0.37725952178239824]


[Sep 10, 15:21:02] [545, 0.37435941010713575]
[Sep 10, 15:21:07] [546, 0.37181699126958845]
[Sep 10, 15:21:11] [547, 0.37304538547992705]
[Sep 10, 15:21:15] [548, 0.3733728152513504]
[Sep 10, 15:21:19] [549, 0.3724276387691498]
[Sep 10, 15:21:23] [550, 0.3735469827055931]
[Sep 10, 15:21:27] [551, 0.3747897747159004]
[Sep 10, 15:21:32] [552, 0.37430131047964094]
[Sep 10, 15:21:36] [553, 0.3762298345565796]
[Sep 10, 15:21:40] [554, 0.37220866471529007]
[Sep 10, 15:21:44] [555, 0.3724040240049362]
[Sep 10, 15:21:48] [556, 0.37570037007331847]
[Sep 10, 15:21:53] [557, 0.37297056764364245]
[Sep 10, 15:21:57] [558, 0.37700504064559937]
[Sep 10, 15:22:01] [559, 0.37350519210100175]
[Sep 10, 15:22:05] [560, 0.3764571049809456]
[Sep 10, 15:22:09] [561, 0.3759055083990097]
[Sep 10, 15:22:13] [562, 0.3725457951426506]
[Sep 10, 15:22:17] [563, 0.37253395438194276]
[Sep 10, 15:22:21] [564, 0.37675503551959993]
[Sep 10, 15:22:26] [565, 0.37538560062646864]
[Sep 10, 15:22:30] [566, 0.3726361459493637

[Sep 10, 15:33:38] [726, 0.3701190412044525]
[Sep 10, 15:33:43] [727, 0.37161120504140854]
[Sep 10, 15:33:47] [728, 0.3716626089811325]
[Sep 10, 15:33:51] [729, 0.371282075047493]
[Sep 10, 15:33:55] [730, 0.37345107555389406]
[Sep 10, 15:33:59] [731, 0.3694102242588997]
[Sep 10, 15:34:03] [732, 0.37424691021442413]
[Sep 10, 15:34:07] [733, 0.37115596830844877]
[Sep 10, 15:34:12] [734, 0.3699056601524353]
[Sep 10, 15:34:16] [735, 0.37174463480710984]
[Sep 10, 15:34:20] [736, 0.3714919951558113]
[Sep 10, 15:34:24] [737, 0.37260346591472626]
[Sep 10, 15:34:28] [738, 0.3705616319179535]
[Sep 10, 15:34:32] [739, 0.3707993787527084]
[Sep 10, 15:34:36] [740, 0.37045524418354037]
[Sep 10, 15:34:41] [741, 0.3711945828795433]
[Sep 10, 15:34:45] [742, 0.36923405796289444]
[Sep 10, 15:34:49] [743, 0.3709836262464523]
[Sep 10, 15:34:54] [744, 0.3706680947542191]
[Sep 10, 15:34:58] [745, 0.372470900118351]
[Sep 10, 15:35:02] [746, 0.37161177664995193]
[Sep 10, 15:35:06] [747, 0.3723784852027893]
[Se

[Sep 10, 15:46:15] [907, 0.3711096143722534]
[Sep 10, 15:46:19] [908, 0.3673281309008598]
[Sep 10, 15:46:23] [909, 0.36937675446271895]
[Sep 10, 15:46:28] [910, 0.37003170818090436]
[Sep 10, 15:46:32] [911, 0.3688602358102798]
[Sep 10, 15:46:36] [912, 0.3733621820807457]
[Sep 10, 15:46:40] [913, 0.3725183054804802]
[Sep 10, 15:46:44] [914, 0.3706510466337204]
[Sep 10, 15:46:48] [915, 0.37152410358190535]
[Sep 10, 15:46:53] [916, 0.3699962520599365]
[Sep 10, 15:46:57] [917, 0.3692979952692986]
[Sep 10, 15:47:01] [918, 0.3731014311313629]
[Sep 10, 15:47:05] [919, 0.3738765373826027]
[Sep 10, 15:47:10] [920, 0.37017098397016523]
[Sep 10, 15:47:14] [921, 0.3716949647665024]
[Sep 10, 15:47:18] [922, 0.3730885660648346]
[Sep 10, 15:47:22] [923, 0.37048459678888324]
[Sep 10, 15:47:26] [924, 0.3679430493712425]
[Sep 10, 15:47:30] [925, 0.37162599682807923]
[Sep 10, 15:47:34] [926, 0.3707479128241539]
[Sep 10, 15:47:38] [927, 0.3690763276815414]
[Sep 10, 15:47:43] [928, 0.3698121291399002]
[Sep